This notebook takes the sentinel 2 lowrez images for a given locID and takes the pixelwise median (removes clouds)
- George Gao 2020

In [67]:
#george gao 28_07_20
#imports 
import numpy as np
import os
import PIL
from PIL import Image
import fnmatch
import csv
import re
from matplotlib import cm
from matplotlib import pyplot as plt
import shutil

In [76]:
s2pth = '/projects/qg35/jlgao2/s2_v2/'
output_pth = '/projects/qg35/jlgao2/s2/'
gt_input = '/projects/qg35/jlgao2/FYP-Sandbox/data/h5/img_wmasks/'
gt_output = '/projects/qg35/jlgao2/s2-gt/'

In [5]:
# create onlyfiles - list with loc ids
onlyfiles = [f for f in os.listdir(s2pth) if os.path.isfile(os.path.join(s2pth, f))]
data = []
for idx, line in enumerate(onlyfiles):
    data.append(line.replace('.tif', ''))
data_list = [e[:15] for e in data]
data_set = set(data_list)

In [62]:
image_matrices = []

for prefix in data_set:
    prefixed = [filename for filename in os.listdir(s2pth) if filename.startswith(prefix)]

    for img in prefixed: #just gets a list o|f photo filenames
        image_matrix = np.array(Image.open(s2pth+img))
        image_matrices.append(image_matrix)

    image_stack = np.concatenate([im[..., None] for im in image_matrices], axis=3)
    median_image = np.median(image_stack, axis=3)
    image_matrices.clear()
    med_im = Image.fromarray((median_image).astype(np.uint8))
    if med_im.size == (60, 60):
        med_im.save(output_pth+prefix+".png")    

In [79]:
onlypngs = [f for f in os.listdir(output_pth) if os.path.isfile(os.path.join(output_pth, f))]

for line in onlypngs:
    gt = Image.open(gt_input+line)
    gt = gt.resize((240, 240))
    gt.save(gt_output+line)
